In [ ]:
import json
import tensorflow as tf
from thesispack.models import ExtendedNN
from thesispack.datasets import ENNdataset, SGAE_dataset
from thesispack.methods import history_figure, pca_denoising_figure
from thesispack.methods import pca_denoising_preprocessing, print_confmtx, list2graph
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
embs_id = '32'
embsD = 32
labels = ['0','1','2']
# all combinations seen (s), unseen (u):
    # {s: ['0','1'], u:['2']}, {s: ['0','2'], u:['1']}, {s: ['1','2'], u:['0']}

seen_labels = ['0','2']
unseen_labels = ['1']
outfilekey = 's-{}-u-{}'.format('-'.join(seen_labels), '-'.join(unseen_labels))
extd = ENNdataset('32', '.')

In [ ]:
knn = KNeighborsClassifier(1)
r_emb = extd.get_r_embs_dict()
X = tf.concat([r_emb[label].reshape(1,-1) for label in labels],axis=0)
Xall = tf.concat([r_emb[key].reshape(1,-1) for key in r_emb.keys()],axis=0)
labels_all = list(r_emb.keys())
Y = extd.get_r_onehot_from_labels([label for label in labels])
knn.fit(X, Y)

In [ ]:
_, A, _ = list2graph(SGAE_dataset['route_links'])
w_p = float(A.shape[1]*A.shape[2] * A.shape[2] - tf.reduce_sum(A)) / tf.reduce_sum(A)
norm = A.shape[1] * A.shape[2] / float((A.shape[1] * A.shape[2] - tf.reduce_sum(A)) * 2)
A = A.shape[1]

In [ ]:
enn = ExtendedNN(embsD,[A.shape[1],64], w_p=w_p, norm=norm, knn=knn,learning_rate=1e-1, early_stop_vars=None)
enn.train(extd.train, 10, extd.val, extd.test, history_learning_process=True)

In [ ]:
save_history_to_file = False
if save_history_to_file:
    with open('./data/extnn/history-{}.json'.format(outfilekey), 'w') as outfile:
        json.dump(enn.history, outfile, indent=4)

In [ ]:
save_obj = ['.','indoor_localization','extnn', 'cost-score', outfilekey]
history_figure(enn.history, save_obj=None)

In [ ]:
enn.set_score_mode(True)
print('val', enn.score_mtr.metric_dataset(extd.val))
print('test', enn.score_mtr.metric_dataset(extd.test))
print('HM', enn.score_mtr.metric_dataset(extd.test)*2*enn.score_mtr.metric_dataset(extd.val)/(enn.score_mtr.metric_dataset(extd.val)+enn.score_mtr.metric_dataset(extd.test)))
enn.set_score_mode(False)

print_confmtx(enn, extd, outfilekey, [1,2])
(_, outs_train, p_train), _, _ = enn.get_results(extd, True)

In [ ]:
pca_vl, pca_ts, pca_emb, knn_pca = pca_denoising_preprocessing(enn, extd, Xall, Y, 1)
save_obj = ['.','indoor_localization','extnn', 'node-emb', outfilekey]
pca_denoising_figure(pca_vl ,pca_ts, pca_emb, knn_pca, labels_all, save_obj=None)